# **재정정보 AI 검색 알고리즘 경진대회 - RAG, RAPTOR 적용**
> **RAG, RAPTOR 적용, Finetuning**

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# 1. 라이브러리 설치하기

In [1]:
# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes

# Transformer 기반 모델과 데이터셋 관련 라이브러리
%pip install transformers[torch] -U
%pip install datasets

# LangChain 관련 라이브러리
%pip install langchain
%pip install langchain_community
%pip install langchain-anthropic
%pip install langchain-huggingface


# 문서 처리 및 문서 임베딩 라이브러리
%pip install pymupdf4llm
%pip install sentence-transformers

# 벡터 데이터베이스 관련 라이브러리
%pip install faiss-cpu

# 데이터
%pip install umap-learn
%pip install scikit-learn
%pip install tiktoken

# env
%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 47.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 53.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━

# 2. 라이브러리 불러오기

In [2]:

import os
import re
import sys
import json
import unicodedata
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import umap
import json
import pymupdf4llm
from typing import Dict, List, Optional, Tuple
import nest_asyncio

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM,
    TextStreamer,
)
from accelerate import Accelerator
from sklearn.mixture import GaussianMixture

from langchain import hub
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import ConfigurableField
import logging
from dotenv import load_dotenv

load_dotenv()
# 비동기 이벤트 루프를 재설정하거나 중첩할 수 있도록 허용
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# CUDA 메모리 할당이 필요할 때마다 메모리 블록을 확장
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

# 3. 초기 설정

In [3]:

class Config:
    def __init__(self, llm_name: str, embedding_name: str, concept: str) -> None:        
        # 모델별 설정 딕셔너리
        self.model_config = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": torch.float16,
                "max_token": 512
            }
        }
        
        # Large Language Model
        self.llm_name = llm_name
        self.llm_config = self.model_config[self.llm_name]
        self.llm_obj = self.setup_llm()
        
        # 임베딩 모델 설정
        self.embedding_name = embedding_name
        self.embedding_obj = self.setup_embedding()
        
        # document
        self.chunk_size = 512
        self.chunk_overlap = 32
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        # Data Path
        self.base_directory = "open/"
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        
        # Output Path
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_name.split('/')[1]}_{concept}_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Langsmith 설정 (환경 변수)
        os.environ["LANGCHAIN_TRACING_V2"] = "true" # os.getenv("LANGCHAIN_TRACING_V2", "false")
        os.environ["LANGCHAIN_API_KEY"] = "" # os.getenv("LANGCHAIN_API_KEY")
        os.environ["LANGCHAIN_PROJECT"] = "Docon-Financial-search" 
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        
    def setup_embedding(self):
        """ 임베딩 모델 설정 """
        embed_id = self.embedding_name
        
        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        embd = HuggingFaceEmbeddings(
            model_name=embed_id,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        store = LocalFileStore("./cache/")
        
        # Cache Embedding 사용
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embd, 
            document_embedding_cache=store, 
            namespace=embed_id
        )
        
        return cached_embeddings
    
    def setup_llm(self):
        """LLM 설정 및 파이프라인 구성"""
        model_id = self.llm_name
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.use_default_system_prompt = False
        
        # 모델 로드 및 양자화 설정 적용
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=self.llm_config["quantization_config"],
            trust_remote_code=True
        )
        
        # 모델을 여러 GPU에 할당
        accelerator = Accelerator()
        model = accelerator.prepare(model)
        
        print(f"#### [ model ] ####\n{model}\n###################")
        
        # 스트리머를 설정하여 토큰이 생성될 때마다 출력
        streamer = TextStreamer(tokenizer)

        # HuggingFacePipeline 객체 생성
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=450,
            streamer=streamer,
        )

        hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

        return hf

# 4. 문서 검색을 위한 RAPTOR 트리 구축

In [4]:

class RaptorClustering:
    def __init__(self, embed_obj, llm_obj):
        self.random_seed = 42
        self.embed_obj = embed_obj
        self.llm_obj = llm_obj
        logger.info(f'[RAPTOR] RaptorClustering 초기화')

    def global_cluster_embeddings(
        self, embeddings: np.ndarray, dim: int, n_neighbors: Optional[int] = None, metric: str = "cosine",
    ) -> np.ndarray:
        """
        임베딩 값에 대해 UMAP을 사용하여 전역 차원에서의 축소 수행
        차원을 'dim'으로 축소하여 클러스터링을 더 쉽게 만듬.
        """
        if n_neighbors is None:
            n_neighbors = int((len(embeddings) - 1) ** 0.5)
        logger.info(f"[RAPTOR] 글로벌 UMAP 차원 축소 수행 - dim={dim}, n_neighbors={n_neighbors}, metric={metric}")
        return umap.UMAP(
            n_neighbors=n_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)
    
    def local_cluster_embeddings(
        self, embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
    ) -> np.ndarray:
        """
        임베딩에 대해 UMAP을 사용하여 지역 차원 축소를 수행. 
        일반적으로 전역 클러스터링 이후에 사용됨.
        """
        logger.info(f"[RAPTOR] 로컬 UMAP 차원 축소 수행 - dim={dim}, num_neighbors={num_neighbors}, metric={metric}")
        return umap.UMAP(
            n_neighbors=num_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)

    def get_optimal_clusters(
        self, embeddings: np.ndarray, max_clusters: int = 50
    ) -> int:
        """
        가우시안 혼합 모델(Gaussian Mixture Model)과 베이지안 정보 기준(BIC)을 사용하여 최적의 클러스터 수를 결정.
        """
        max_clusters = min(max_clusters, len(embeddings))  # 최대 클러스터 수와 임베딩의 길이 중 작은 값을 최대 클러스터 수로 설정
        n_clusters = np.arange(1, max_clusters)            # 1부터 최대 클러스터 수까지의 범위를 생성
        bics = []                                          # BIC 점수를 저장할 리스트
        
        logger.info("[RAPTOR] Gaussian Mixture Model(GMM)을 사용하여 최적의 클러스터 수 결정 중")
        for n in n_clusters: # 각 클러스터 수에 대해 반복
            gm = GaussianMixture(n_components=n, random_state=self.random_seed) # 가우시안 혼합 모델 초기화
            gm.fit(embeddings)              # 임베딩에 대해 모델 학습
            bic = gm.bic(embeddings)        # 학습된 bic 점수 추가
            bics.append(bic)
            logger.debug(f"[RAPTOR] {n}개의 클러스터에 대한 BIC: {bic}")
        
        optimal_clusters = n_clusters[np.argmin(bics)] # BIC 점수가 가장 낮은 클러스터 수를 반환
        logger.info(f"[RAPTOR] 결정된 최적의 클러스터 수: {optimal_clusters}")
        return optimal_clusters  

    def GMM_cluster(
        self, embeddings: np.ndarray, threshold: float
    ) -> Tuple[List[np.ndarray], int]:
        """
        임베딩을 GMM을 사용해 클러스터링합니다. 주어진 확률 임계값을 기준으로 클러스터링 실행
        """
        logger.info("[RAPTOR] GMM을 사용하여 임베딩 클러스터링 중")
        n_clusters = self.get_optimal_clusters(embeddings)  # 최적의 클러스터 수를 구합니다.
        gm = GaussianMixture(n_components=n_clusters, random_state=self.random_seed) # 가우시안 혼합 모델을 초기화합니다.
        gm.fit(embeddings)                                         # 임베딩에 대해 모델을 학습합니다.
        probs = gm.predict_proba(embeddings)                       # 임베딩이 각 클러스터에 속할 확률을 예측합니다.
        labels = [np.where(prob > threshold)[0] for prob in probs] # 임계값을 초과하는 확률을 가진 클러스터를 레이블로 선택합니다.
        logger.info(f"[RAPTOR] 임계값 {threshold}을 기준으로 클러스터 레이블 생성 완료")
        return labels, n_clusters                                  # 레이블과 클러스터 수(튜플)를 반환합니다.

    def perform_clustering(
        self, embeddings: np.ndarray, dim: int, threshold: float,
    ) -> List[np.ndarray]:
        """
        차원 축소 -> GMM을 사용한 클러스터링 -> 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행
        """
        logger.info("[RAPTOR] 계층적 클러스터링 과정 시작")
        if len(embeddings) <= dim + 1:
            logger.warning("[RAPTOR] 데이터가 충분하지 않음 - 모든 데이터 포인트에 대해 단일 클러스터 반환")
            return [np.array([0]) for _ in range(len(embeddings))]

        # 글로벌 차원 축소
        reduced_embeddings_global = self.global_cluster_embeddings(embeddings, dim)
        # 글로벌 클러스터링
        global_clusters, n_global_clusters = self.GMM_cluster(
            reduced_embeddings_global, threshold
        )

        all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
        total_clusters = 0

        # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
        for i in range(n_global_clusters):
            # 현재 글로벌 클러스터에 속하는 임베딩 추출
            global_cluster_embeddings_ = embeddings[
                np.array([i in gc for gc in global_clusters])
            ]

            if len(global_cluster_embeddings_) == 0:
                continue
            if len(global_cluster_embeddings_) <= dim + 1:
                # 작은 클러스터는 직접 할당으로 처리
                local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
                n_local_clusters = 1
            else:
                # 로컬 차원 축소 및 클러스터링
                reduced_embeddings_local = self.local_cluster_embeddings(
                    global_cluster_embeddings_, dim
                )
                local_clusters, n_local_clusters = self.GMM_cluster(
                    reduced_embeddings_local, threshold
                )

            # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
            for j in range(n_local_clusters):
                local_cluster_embeddings_ = global_cluster_embeddings_[
                    np.array([j in lc for lc in local_clusters])
                ]
                indices = np.where(
                    (embeddings == local_cluster_embeddings_[:, None]).all(-1)
                )[1]
                for idx in indices:
                    all_local_clusters[idx] = np.append(
                        all_local_clusters[idx], j + total_clusters
                    )

            total_clusters += n_local_clusters
            logger.info(f"[RAPTOR] 글로벌 클러스터 {i} 처리 후 총 클러스터 수: {total_clusters}")

        return all_local_clusters


    def embed(self, texts: List[str]) -> np.ndarray:
        """
        텍스트 문서 목록에 대한 임베딩을 생성.
        embd 객체는 텍스트 목록을 받아 그 임베딩을 반환하는 `embed_documents` 메소드를 가지고 있음.
        """        
        text_embeddings = self.embed_obj.embed_documents(texts)  # 텍스트 문서들의 임베딩을 생성합니다.
        text_embeddings_np = np.array(text_embeddings)  # 임베딩을 numpy 배열로 변환합니다.
        return text_embeddings_np                       # 임베딩된 numpy 배열을 반환합니다.


    def embed_cluster_texts(self, texts: List[str]) -> pd.DataFrame:
        """
        텍스트 목록을 임베딩하고 클러스터링하여, 텍스트, 임베딩, 그리고 클러스터 라벨이 포함된 DataFrame을 반환합니다.
        이 함수는 임베딩 생성과 클러스터링을 단일 단계로 결합합니다.
        """
        logger.info("[RAPTOR] 텍스트 임베딩 및 클러스터링 시작")
        text_embeddings_np = self.embed(texts) # 임베딩 생성
        cluster_labels = self.perform_clustering(
            text_embeddings_np, 10, 0.1
        )  # 임베딩에 대해 클러스터링 수행
        df = pd.DataFrame()                   # 결과를 저장할 DataFrame 초기화
        df["text"] = texts                    # 원본 텍스트 저장
        df["embd"] = list(text_embeddings_np) # DataFrame에 리스트로 임베딩 저장
        df["cluster"] = cluster_labels        # 클러스터 라벨 저장
        logger.info("[RAPTOR] 클러스터링 완료 - 클러스터가 포함된 DataFrame 생성 완료")
        return df


    def fmt_txt(self, df: pd.DataFrame) -> str:
        """
        DataFrame에 있는 텍스트 문서를 단일 문자열로 포맷합니다.
        """
        logger.info("[RAPTOR] DataFrame의 텍스트를 단일 문자열로 포맷하는 중")
        unique_txt = df["text"].tolist()
        return "--- --- \n --- --- ".join(unique_txt)  # 텍스트 문서들을 특정 구분자로 결합하여 반환


    def embed_cluster_summarize_texts(
        self, texts: List[str], level: int
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        텍스트 목록에 대해 임베딩, 클러스터링 및 요약을 수행합니다. 
        """
        # 텍스트를 임베딩하고 클러스터링하여 'text', 'embd', 'cluster' 열이 있는 데이터프레임을 생성합니다.
        logger.info(f"[RAPTOR] 텍스트 요약 시작 - 레벨 {level}")
        df_clusters = self.embed_cluster_texts(texts)
        expanded_list = []

        # 데이터프레임 항목을 문서-클러스터 쌍으로 확장하여 처리를 간단하게 합니다.
        for index, row in df_clusters.iterrows():
            for cluster in row["cluster"]:
                expanded_list.append(
                    {"text": row["text"], "embd": row["embd"], "cluster": cluster}
                )

            # 확장된 목록에서 새 데이터프레임을 생성합니다.
        expanded_df = pd.DataFrame(expanded_list)

        # 처리를 위해 고유한 클러스터 식별자를 검색합니다.
        all_clusters = expanded_df["cluster"].unique()

        logger.info(f"--Generated {len(all_clusters)} clusters--")

        # 요약
        template = """
        여기 중앙정부 재정 정보와 관련된 문서의 하위 집합이 있습니다.
        재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 데이터가 주어집니다.
        
        제공된 문서의 자세한 요약을 제공하십시오.
        
        문서:
        {context}
        """
        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | self.llm_obj | StrOutputParser()

        # LLM을 사용하여 요약문을 생성합니다.       
        # 각 클러스터를 처리하고 요약을 생성합니다.
        summaries = []
        for i in all_clusters:
            df_cluster = expanded_df[expanded_df["cluster"] == i]
            formatted_txt = self.fmt_txt(df_cluster)
            summaries.append(chain.invoke({"context": formatted_txt}))
            
        # 요약, 해당 클러스터 및 레벨을 저장할 데이터프레임을 생성합니다.
        df_summary = pd.DataFrame(
            {
                "summaries": summaries,
                "level": [level] * len(summaries),
                "cluster": list(all_clusters),
            }
        )
        logger.info(f'클러스터링 및 요약 결과 : \ndf_clusters : \n{df_clusters}\n\n\ndf_summary\n{df_summary}')
        return df_clusters, df_summary

    def recursive_embed_cluster_summarize(
        self, texts: List[str], level: int = 1, n_levels: int = 3
    ) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
        """
        지정된 레벨까지 또는 고유 클러스터의 수가 1이 될 때까지 텍스트를 재귀적으로 임베딩, 클러스터링, 요약하고 각 레벨에서의 결과를 저장.
        """
        results = {}

        # 현재 레벨에 대해 임베딩, 클러스터링, 요약 수행
        logger.info(f"[RAPTOR] 레벨 {level}에서 임베딩, 클러스터링 및 요약 시작")
        df_clusters, df_summary = self.embed_cluster_summarize_texts(texts, level)

        # 현재 레벨의 결과 저장
        results[level] = (df_clusters, df_summary)

        # 추가 재귀가 가능하고 의미가 있는지 결정
        unique_clusters = df_summary["cluster"].nunique()
        if level < n_levels and unique_clusters > 1:
            logger.info(f"[RAPTOR] 재귀 조건 만족 - 다음 레벨 {level + 1}로 이동")
            new_texts = df_summary["summaries"].tolist()
            next_level_results = self.recursive_embed_cluster_summarize(
                new_texts, level + 1, n_levels
            )

            # 다음 레벨의 결과를 현재 결과 사전에 병합
            results.update(next_level_results)
        
        logger.info(f'results : \n{results}')
        return results

# 5. 데이터셋 전처리 및 벡터 DB 구축

In [5]:

class DataIngestionPipeline:
    def __init__(self, config:Config, raptor:RaptorClustering, embed_obj):
        self.config = config
        self.raptor = raptor
        logger.info("PDFProcessor 초기화 완료.")
                
    def _normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def _preprocess_text(self, text: str) -> str:
        """ 텍스트 전처리 함수 """
        # 불필요한 공백 제거
        text = text.strip()
        # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', text) # gemma
        # 공백 처리 및 불필요한 줄바꿈 제거
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def _process_pdf(self, file_path) -> List[str]:
        """ PDF 파일 로드, 텍스트 추출 """
        logger.info(f"PDF 처리 중: {file_path}")
        docs_md_content = pymupdf4llm.to_markdown(file_path)
        
        # 전처리
        preprocess_content = self._preprocess_text(docs_md_content)
        
        # headers_to_split_on = [
        #     ("#", "Header 1"),
        #     ("##", "Header 2"),
        #     ("###", "Header 3"),
        # ]
        
        # md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
        # md_chunks = md_header_splitter.split_text(md_content)
        
        # 텍스트 데이터를 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=self.config.chunk_size, chunk_overlap=self.config.chunk_overlap
        )
        chunks = text_splitter.split_text(preprocess_content)
        logger.info(f"텍스트를 {len(chunks)}개의 청크로 분할 완료.")
        
        return chunks
    
    def init_retriever(self, df):
        """ PDF 데이터를 처리하고 벡터 데이터베이스 및 리트리버를 초기화 """
        pdf_databases = {}
        unique_paths = df['Source_path'].unique()

        for path in tqdm(unique_paths, desc="Processing PDFs"):
            # 경로 정규화 및 절대 경로 생성
            normalized_path = self._normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

            # pdf -> 벡터 DB 구축
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            print(f"PDF -> 벡터 DB 구축 중 : [{pdf_title}]...")
        
            # RAPTOR 트리 구축
            leaf_texts = self._process_pdf(full_path) # list[str]
            raptor_results = self.raptor.recursive_embed_cluster_summarize(
                leaf_texts, level=1, n_levels=3
            )
            logger.info(f"RAPTOR 트리 구축 완료: {pdf_title}.")
        
            # 각 레벨의 요약을 추출하여 all_texts에 추가
            all_texts = leaf_texts.copy()
            for level in sorted(raptor_results.keys()):
                summaries = raptor_results[level][1]["summaries"].tolist()  # 현재 레벨의 DataFrame에서 요약을 추출합니다.
                all_texts.extend(summaries)                                 # 현재 레벨의 요약을 all_texts에 추가합니다.
            logger.info(f"{pdf_title}에 대해 요약 추가 완료.")
        
            # FAISS 벡터 데이터베이스 구축
            vector_store = FAISS.from_texts(texts=all_texts, embedding=self.config.embedding_obj)
            raptor_retriever = vector_store.as_retriever(search_kwargs={"k": 3}).configurable_fields(
                search_kwargs = ConfigurableField(
                    id='search_kwargs_faiss',
                    name='Search Kwargs',
                    description='The search kwargs to use',
                )
            )
            
            pdf_databases[pdf_title] = {
                'db': vector_store,
                'retriever' : raptor_retriever,
            }
        
        logger.info("모든 PDF 처리 완료.")
        return pdf_databases
    

# 6. QA 생성 모듈

In [ ]:
""" QA 프롬프트 템플릿 """

############ ChatPromptTemplate 사용 ###############3
# system_prompt_template = """ 
# 당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
# 답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

# 아래 조건을 지켜서 답변을 생성해주세요 :
# 주어와 서술어를 사용하여 온전한 문장을 완성시키세요.
# 새로운 질문을 생성하지 마세요.
# 아래 정보의 내용을 참고해서 질문에 정확한 답변을 생성하세요.
# 기호나 공백을 제거하고 깔끔하게 답변하세요.

# 주어진 정보는 다음과 같습니다. : 
# {context}
# """
# financial_qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt_template),
#         ("human", "{input}"),
#     ]
# )

############ PromptTemplate 사용 ###############3
template = """ 
당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

주어진 정보는 다음과 같습니다. : 
{context}

질문 :
{question}

새로운 질문을 생성하지 마세요.
주어진 정보의 내용을 참고해서 질문에 정확한 답변을 생성하세요.
기호나 공백을 제거하고 깔끔한 답변을 생성하세요.
주어와 서술어가 포함된 문장을 완성시켜서 답변을 생성하세요.
답변을 생성할 때 "답변:", "<eos>"와 같은 단어를 포함시키지 마세요.
"""
financial_qa_prompt = PromptTemplate.from_template(template) 


############ 다른 프롬프트 ###############3
# system_prompt_template = """
# 당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.
# 답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.

# 주어진 질문에 대해 다음 단계에 따라 답변하세요:
# 1. 질문을 이해하고 필요한 재정 개념을 설명합니다.
# 2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.
# 3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.

# 답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.
# 단, 주어와 서술어를 적절히 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

# 다음 정보를 바탕으로 질문에 답하세요 :
# {context}            
# """

In [15]:

class FISearch:
    def __init__(self, config, prompts):
        self.config = config
        self.prompts = prompts
    
    def _normalize_string(self, s):
        """유니코드 정규화"""
        return unicodedata.normalize('NFC', s)
    
    def _format_docs(self, docs):
        # 문서의 페이지 내용을 이어붙여 반환합니다.
        return "\n\n".join(doc.page_content for doc in docs)
    
    # def _clean_answer(self, answer):
    #     # 다양한 불필요한 텍스트 패턴을 제거
    #     patterns_to_remove = [
    #         r"^\s*###? Response:\s*",
    #         r"^\s*Response:\s*",
    #         r"^\s*AI:\s*",
    #         r"^\s*Quad:\s*",
    #         r"^\s*\|\-\s*\n?\s*\n?\s*###? Response:\s*",
    #         r"^\s*\|\-\s*\n?\s*\n?\s*Response:\s*",
    #         r"^\s*\|\-\s*\n?\s*\n?\s*AI:\s*",
    #         r"^\s*\|\-\s*\n?\s*\n?\s*Quad:\s*",
    #         r"<eos>\s*$"
    #     ]
    #     for pattern in patterns_to_remove:
    #         answer = re.sub(pattern, "", answer)
    #     return answer.strip()
        
    def generate_answers(self, df, pdf_databases):
        """DataFrame의 각 질문에 대해 답변을 생성"""
        results = []
        
        # DataFrame의 각 행에 대해 처리
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            # 소스 문자열 정규화
            source = self._normalize_string(row['Source'])
            question = row['Question']

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self._normalize_string(k): v for k, v in pdf_databases.items()}
            raptor_retriever = normalized_keys[source]['retriever']
            
            # # RAG 체인 정의
            rag_chain = (
                {"context": raptor_retriever | self.format_docs, "question": RunnablePassthrough()}
                | self.prompts
                | self.config.llm_obj
                | StrOutputParser()
            )
            
            # # Retrieval Chain 생성
            # documents_chain = create_stuff_documents_chain(self.config.llm_obj, prompt=self.prompts)
            # rag_chain = create_retrieval_chain(ensemble_retriever, documents_chain) # EnsembleRetreiver + document chain
            
            
            
            # 답변 추론
            print(f"Question: {question}")
            # result = rag_chain.invoke({"input": query}) # Retreival Chain 사용
            result = rag_chain.invoke(question)
            print(f"답변: {result}\n================================================")
            # result = self._clean_answer(result)
            # print(f"후처리 답변: {result}\n================================================")
            
            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": question,
                "Answer": result
            })

        return results

---

# 7. 최종 통합, 실행 및 저장

In [16]:
if __name__ == "__main__":
    config = Config(
        llm_name="mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1", 
        embedding_name="intfloat/multilingual-e5-large",
        concept="financial_analysis"
    )
    
    # 데이터 불러오기
    df = pd.read_csv(config.test_csv_path)
    
    # RaptorClustering 객체 초기화
    raptor = RaptorClustering(
        embed_obj=config.embed_model_obj,
        llm_obj=config.llm_obj
    )
    
    # 파이프라인 초기화
    dipipeline = DataIngestionPipeline(config=config, raptor=raptor)
    
    # 벡터 데이터베이스 및 리트리버 초기화
    pdf_databases = dipipeline.init_retriever(df)
    
    # 검색 및 답변 생성
    search_engine = FISearch(config=config, prompts=financial_qa_prompt)
    results = search_engine.generate_answer(df, pdf_databases)
    
    # 제출 샘플 파일 로드 및 답변 추가
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")

    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)
    
    print("결과가 성공적으로 저장되었습니다.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


#### [ model ] ####
Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        

INFO:__main__:[RAPTOR] RaptorClustering 초기화
INFO:__main__:PDFProcessor 초기화 완료.
Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_중소벤처기업부_혁신창업사업화자금(융자).faiss
INFO:__main__:PDF 처리 완료: 중소벤처기업부_혁신창업사업화자금(융자).
INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_보건복지부_부모급여(영아수당) 지원.faiss
INFO:__main__:PDF 처리 완료: 보건복지부_부모급여(영아수당) 지원.
INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_보건복지부_노인장기요양보험 사업운영.faiss
INFO:__main__:PDF 처리 완료: 보건복지부_노인장기요양보험 사업운영.
INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_산업통상자원부_에너지바우처.faiss
INFO:__main__:PDF 처리 완료: 산업통상자원부_에너지바우처.
INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_국토교통부_행복주택출자.faiss
INFO:__main__:PDF 처리 완료: 국토교통부_행복주택출자.
INFO:root:기존 벡터 DB 로드 중: ./RAPTOR_「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.faiss
INFO:__main__:PDF 처리 완료: 「FIS 이슈 &

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
<bos>System: 
            당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.
            답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.
            
            주어진 질문에 대해 다음 단계에 따라 답변하세요:
            1. 질문을 이해하고 필요한 재정 개념을 설명합니다.
            2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.
            3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.
            
            답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.
            단, 주어와 서술어를 적절히 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.
            
            다음 정보를 바탕으로 질문에 답하세요 :
            
        2024년 창업기반지원의 사업비는 얼마이며, 어떤 방식으로 지원되는가?

        2024년 창업기반지원의 사업비는 1조 7,958억원이며, 융자를 통해 지원된다.

## 사  업  명   혁신창업사업화자금(융자) (5152-301)   ## 1. 사업 코드 정보  |구분|기금|소관|실국(기관)|계정|분야|부문| |---|---|---|---|---|---|---| |코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118| |명칭|||||산업·중소기업 및 에너지|창업 및 벤처|   |구분|프로그램|단위사업|세부사업| |---|---|---|---| |코드|5100|5152|301| |명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|   ## 구분

...

Answer: {'input': '국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?', 'context': [Document(page_content='**FIS** **ISSUE & FOCUS**  **(우발부채 인식) 국가회계기준(중앙정부)과 지방회계기준(지방정부)에서 우발부채 또는 우발상황 등을 정의**  ‣ \x07중앙정부와 지방정부는 각각 「국가회계기준에 관한 규칙」과 「지방자치단체 회계기준에 관한 규칙」에서 우  발부채 또는 우발상황 등을 정의하고, 이를 근거로 인식한 우발부채 현황을 결산보고서 주석 등에 공시[8)]  -  \x07「국가회계기준에 관한 규칙」은 제50조에서 ‘충당부채’, ‘우발부채’ 및 ‘우발자산’을 정의하고 있으며, 관련 예  규로서 「충당부채, 우발부채, 우발자산 회계처리지침」 운영  -  \x07「지방자치단체 회계기준에 관한 규칙」은 제59조에서 ‘우발상황’, ‘우발손실’ 및 ‘우발이익’으로 정의하고 있  으며, 지방정부의 우발부채의 회계처리가 중앙정부와 상이[9)]  ‣ \x07국가회계기준의 우발부채 정의는 한국채택국제회계기준(K–IFRS) 또는 국제공공부문회계기준(IPSAS) 등  을 준용하고 있으나 광의적 개념 정의에 머무르는 한계 **[10)]**  -  \x07「국가회계기준에 관한 규칙」은 2018년 일부개정으로 제50조의 우발상황, 우발이익 및 우발손실 용어를 삭제  하고 충당부채, 우발부채 및 우발자산 용어를 신설하여 인식 및 측정기준을 국제공공부문회계기준(IPSAS) 등  타 회계기준과 비슷하게 개선  -  \x07이에 대한 회계처리지침을 신설하는 등 여러 노력이 있어 왔으나, 공시 현황을 보면 여전히 항목들을 주석5  ‘우발사항 및 약정사항’ 내에 열거하는 수준에 머물러 있음  -  \x07국제기준 GFSM2014는 우발부채를 ‘명시적(explicit)’ 및 ‘암묵적(implicit)’ 성격으로 구분하고 그 범주에 대  해 논의하고 있는데, 이에 맞춰 우발부채 용어 및

Answering Questions:  99%|█████████▉| 97/98 [10:58<00:09,  9.51s/it]

Answer: {'input': '우발부채란 무엇이며, 그 관리가 왜 중요한가?', 'context': [Document(page_content='**들어가며**  ## 01  **ISSUE** **왜 우발부채에 주목하는가?**  **ISSUE**   ‣ \x07국제기준 재정통계 산출에 발생주의(accrual basis)[1)] 회계기준이 적용되면서 미래의 다양한 의제의무  (constructive obligation)[2)]를 포괄하는 우발부채 관리에 대한 중요성 인식  -  \x07기업회계에서는 우발부채 규모가 그리 크지 않아 중요한 요소가 아니지만, 공공부문에서는 재정건전성, 재정  위기 관리 등 다양한 관점에서 중요하게 다뤄져야 할 필요  -  \x07GFSM2001에서 광의적 개념으로 정의하였던 것에 비해, GFSM2014에서는 ‘명시적 우발부채’와 ‘암묵적 우  발부채’로 세분화하여 우발부채 범주를 하위 항목까지 구체적으로 다룸 **[3)]**  ‣ \x07우리나라는 2011년부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있으나, 용어  사용에 혼란이 있고 분류상 오류가 있는 등 체계적 관리 대안에 대한 논의가 필요한 시점  -  \x07국제적으로 정부의 채무지속가능성분석(DSA)[4)]을 위한 기초자료로 활용되는 등 주요하게 인식되는 추세와  비교해 우리나라에서는 아직까지 체계적으로 다뤄지지 않고 있음  **FOCUS** **우발부채 관련 주요 쟁점은?**  **FOCUS**   ‣ \x07우발부채 개념 및 용어 사용의 혼란, 우발부채 분류기준 재검토, 새로운 분류기준 정립  -  \x07발생주의 회계기준을 적용한 국가결산보고서가 10년간 작성되었으나 우발부채의 인식과 공시는 크게 달라지지  않음(현재 주석에서 ‘우발사항 및 약정사항’으로 보고)  -  \x07우리나라는 국가회계기준, 지방회계기준, 한국채택국제회계기준(K–IFRS) 등에서 우발부채를 광의적 개념에  서만 정의하고 있어 용어 사용 혼란 및 체계적 관리 한계(본고 

Answering Questions: 100%|██████████| 98/98 [11:12<00:00,  6.86s/it]

Answer: {'input': '보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?', 'context': [Document(page_content='**18) \x07보증(guarantees)은 명시적 우발부채의 가장 보편적 형태이나 모든 보증이 우발부채에 해당하는 것은 아니다. 예컨대 ‘파생금융상품 형태의 보증’이나 ‘표준화된 보증** 제도하에서 설정하는 충당부채 형태의 보증’은 재정상태표에 부채로 기록한다.\n **19) \x07신용부도 스왑(CDS) 등 파생금융상품을 통해 제공되는 보증은 (다른 파생금융상품과 마찬가지로) 금융자산 및 부채(liabilities)이나 채무(debt)에는 포함되지 않는다.** 하지만 이들 계약이 공공부문 단위의 부채에 포함되고 상당한 손실이 발생할 수 있으므로 파생금융상품 정보를 채무통계와 함께 기록할 것을 권고하고 있다.\n **20) \x07표준화 보증(standardized guarantees)은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증을 말하며, 수출(무역)신용 보증, 환보증, 다양한** 종류의 보험(예금, 농작물, 자연재해 등), 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 있다. 주로 유사한 특징을 지니고 위험 분산을 목적으로 하는 보증들이 많으며 보증인은 확률통계에 기반하여 평균손실(채무불이행률)을 추정할 수 있는데, 이러한 채무불이행률로 인해 공공부문의 우발부채가 아니라 표준화 보증 충당부채로 인식한다.\n **21) \x07잠재적 법적청구권은 계류 중인 법정 소송사건에서 비롯되는 청구권을, 배상금(보상금)은 다른 당사자가 입을 수 있는 손실 또는 손상의 위험을 수용하는 약속으로,** 예를 들어 정부와 기타 단위 간의 계약에서 발생하는 예기치 못한 조세채무에 따른 배상을, 미불입 자본은 주주로 있는 실체(국제금융기관 등)의 요구에 따라 추가 자본을 제공해야 할 의무를 말한다.\n \n-----')

---

## 틀린 답안 분석

1. **TEST_005**: "2015년 1월입니다." 
<br>→ "재창업자금이 재도약지원자금으로 이관된 연도는 2015년입니다."는 맞지만, 1월이라는 정보는 주어지지 않았습니다.

2. **TEST_008**: "사업계획수립/공고 ⇨ [사전상담 및 신청][‧][접수] ⇨ [‧] (중기부, 중진공) (중소기업 → 중진공) (중진공 → 중소기업)..." 
<br>→ 이 답변은 올바르지 않으며, 복잡하게 작성되어 있습니다. 올바른 절차는 "사업계획 수립 및 공고 (중기부, 중진공), 사전상담 및 신청·접수 (중소기업 → 중진공), 서류 및 현장실사 (중진공 → 중소기업), 지원결정통보 (중진공 → 중소기업), 융자 실행 (중진공, 은행 → 중소기업)"입니다.

3. **TEST_013**: "부모급여 지원사업은 2022년 7월부터 시행되었습니다." 
<br>→ 부모급여 지원사업은 2023년 1월부터 시행되었습니다.

4. **TEST_015**: "노인장기요양보험 사업의 목적은 장기요양보험가입자의 자격취득·상실, 장기요양보험료의 납부·징수 및 결손처분 등을 통해 국민의 건강한 노후생활 보장에 기여하는 것이다." 
<br>→ 노인장기요양보험 사업의 목적은 고령이나 노인성 질병으로 인해 일상생활을 혼자서 수행하기 어려운 노인 등에게 신체 또는 가사 활동을 제공하여 노후의 건강증진과 생활 안정을 도모하는 것입니다.

5. **TEST_021**: "장기요양기관 지정갱신제의 법적 근거는 2025년에 마련되었습니다." 
<br>→ 장기요양기관 지정갱신제의 법적 근거는 2018년에 마련되었습니다.

6. **TEST_023**: "에너지 바우처는 2015년부터 시행된 제도로, 난방비 부담이 큰 저소득층 등에게 에너지 사용료를 지원하는 것입니다." 
<br>→ 이 문장은 정답이 아닙니다. 에너지 바우처 제도는 하절기와 동절기에 에너지 이용에 필요한 비용을 지원하는 제도로, 구체적으로 전기요금, 연료비 등을 지원합니다.

7. **TEST_025**: "한국에너지공단과 산업통상자원부" 
<br>→ 2024년 에너지바우처 사업의 사업시행주체는 한국에너지공단과 한국광해광업공단입니다.

8. **TEST_031**: "2023년 에너지바우처 사업 예산에서 에너지바우처 전달체계 구축에 11,048백만원이 할당되었습니다." 
<br>→ 올바른 할당액은 37.5백만원입니다.

9. **TEST_038**: "공공주택 특별법 제3조의2(공공주택의 재원ㆍ세 재지원 등)" 
<br>→ "행복주택출자 사업은 주택도시기금법 제9조 제1항 가목 및 공공주택특별법 제2조 1호 가목에 근거하여 추진됩니다."가 올바른 근거입니다.

10. **TEST_039**: "행복주택출자 사업은 서민 주거안정을 위한 주택시장 정상화 종합대책의 일환으로, 2013년 13만호 공급 계획을 발표하여 시작되었습니다." 
<br>→ 올바른 답변은 "행복주택출자 사업은 국민의 행복주거 실현을 위한 보편적 주거복지 정책의 일환으로 시행됩니다."입니다.

---

## 수정 및 개선 제안

1.	PDF 텍스트 추출 및 전처리 개선:
    - 텍스트 구조 보존: pymupdf4llm.to_markdown에서 텍스트를 추출할 때, 문단, 표, 리스트 등의 구조를 가능한 한 유지하도록 개선합니다. 텍스트를 처리할 때 불필요한 줄바꿈을 제거하되, 의미 있는 구분자를 기준으로만 제거하도록 수정합니다.
	- 청크 크기 조정: chunk_size_tok의 크기를 2000에서 512 정도로 줄여서 청크 크기를 조정합니다. 이렇게 하면 문서의 의미 있는 부분을 더 잘 보존할 수 있습니다.

2.	임베딩과 클러스터링 전략 수정:
	- 차원 축소와 클러스터링 개선: UMAP의 n_neighbors 파라미터를 조정하여 데이터의 밀집도를 고려하고, GMM 클러스터링 전에 샘플링을 도입하여 더 나은 클러스터링 결과를 얻을 수 있습니다. 또한, 클러스터링 후 클러스터 간 유사성을 재평가하여 의미 없는 클러스터를 통합할 수 있는 방안을 고려합니다.
	- 클러스터의 의미 검토: 각 클러스터의 문서 내용을 확인하여 실제로 의미 있는 클러스터링이 이루어졌는지 검토합니다. 클러스터가 의미 없는 경우, 클러스터링 전략을 변경하거나 청크 크기를 조정하는 등 추가 조정이 필요합니다.

3.	LLM 기반 답변 생성 개선:
	- 컨텍스트 강화: 질문에 대한 컨텍스트를 좀 더 구체적으로 제공하기 위해 시스템 프롬프트를 조정합니다. 예를 들어, 해당 질문이 특정 법령, 연도, 사업명 등에 대한 질문인 경우 이를 명시적으로 포함시킵니다.
	- 프롬프트 세분화: 질문 유형에 따라 프롬프트를 세분화하여 보다 정교한 답변을 유도할 수 있습니다. 예를 들어, 법령 근거를 묻는 질문과 예산 추이를 묻는 질문에 대해 각각 최적화된 프롬프트를 사용할 수 있습니다.
	- 검색 방식 개선: 중요한 정보를 검색할 때 단순한 벡터 검색이 아닌, 질문 유형에 따라 다르게 접근합니다. 예를 들어, 법률 조항에 관한 질문에는 법률 텍스트 검색을, 예산에 관한 질문에는 수치 데이터 검색을 우선적으로 수행합니다.

4.	RAPTOR의 적합성 검토:
	- RAPTOR 적합성: RAPTOR 클러스터링 및 요약 전략은 복잡한 대규모 문서에서 의미 있는 요약을 생성하는 데 효과적일 수 있습니다. 그러나 이 접근 방식은 소규모 법률 문서 또는 정책 문서에서는 오히려 불필요하게 복잡할 수 있습니다. 이러한 경우, RAPTOR 대신 직접적인 검색과 요약 방법을 사용하는 것이 더 나을 수 있습니다. 예를 들어, 특정 텍스트에서 직접적인 답변을 찾고 이를 요약하는 방식이 더 적합할 수 있습니다.

## 최적화 제안

1.	프롬프트 전략 변경:
	- 프롬프트를 질문의 유형에 맞춰 조정합니다. 예를 들어, 법적 근거에 대한 질문에 대해 법령 텍스트를 먼저 검색하도록 하고, 그런 다음 해당 법령 조항을 프롬프트에 포함시켜 답변을 생성하도록 할 수 있습니다.

2.	PDF 텍스트 처리 최적화:
	- PDF 텍스트 추출 시, 중요한 문단이나 항목을 잃지 않도록 전처리 방식을 더 세밀하게 조정합니다. 특히 표나 리스트 형식의 정보가 손실되지 않도록 주의합니다.

3.	RAPTOR 대신 직접 검색 방식 도입:
	- RAPTOR의 사용을 최소화하고, 특정 질문에 대해서는 직접적으로 관련 텍스트를 검색하여 LLM에 제공하는 방식을 고려합니다. 이를 통해 불필요한 클러스터링 단계에서 발생할 수 있는 오류를 줄일 수 있습니다.

---

정확한 정보 검색이 중요한 상황에서는 다양한 정보 검색 기술과 알고리즘을 조합하여 최적의 결과를 도출할 수 있습니다. EnsembleRetriever는 이러한 상황에서 효과적으로 작동할 수 있으며, 이를 기반으로 추가적으로 시도해볼 만한 기술이나 알고리즘은 다음과 같습니다:

### 1. **EnsembleRetriever**
   - **설명**: EnsembleRetriever는 여러 검색 모델을 조합하여 결과를 통합하는 방식으로, 개별 모델이 가지는 장점을 최대화하고 단점을 보완합니다. 각 검색 모델이 서로 다른 특성을 가진 문서에서 강점을 발휘하기 때문에, 이러한 모델들을 조합하면 전체적인 검색 성능이 향상될 수 있습니다.
   - **시도할 수 있는 조합**:
     - **BM25 + Dense Retriever**: 전통적인 BM25와 Dense Retriever (예: DPR, ColBERT)를 조합하여 결과의 다양성을 확보.
     - **Dense Retriever + Reranker**: Dense Retriever로 초기 검색을 수행하고, BERT 기반의 Reranker로 결과를 재정렬하여 정확도를 높임.

### 2. **ColBERT (Contextualized Late Interaction over BERT)**
   - **설명**: ColBERT는 BERT 기반의 문맥화된 표현을 사용하여 빠르고 효율적인 검색을 가능하게 하는 모델입니다. Late Interaction 메커니즘을 통해 문서와 쿼리의 세부적인 일치도를 계산합니다.
   - **적용 가능성**: 대규모 문서 집합에서 쿼리에 대한 세밀한 일치도를 평가해야 하는 경우 유용합니다. 정확한 검색을 위해 다양한 쿼리 변형을 지원합니다.

### 3. **DPR (Dense Passage Retrieval)**
   - **설명**: DPR은 BERT를 기반으로 한 Dense Retriever로, 쿼리와 문서를 동일한 벡터 공간으로 매핑하여 효율적인 검색을 수행합니다. 대규모 데이터셋에서 정확한 검색 성능을 발휘합니다.
   - **적용 가능성**: 대규모의 비구조화된 문서에서 관련성을 높은 정확도로 판단할 수 있습니다. 비슷한 맥락의 문서를 찾을 때 매우 유용합니다.

### 4. **BM25 + Transformer-based Reranker**
   - **설명**: BM25와 같은 전통적인 정보 검색 모델을 사용하여 초기 검색을 수행한 후, Transformer 기반의 Reranker (예: BERT Reranker)로 결과를 재정렬합니다. 이는 초기 검색에서 놓칠 수 있는 문서들을 후처리 단계에서 보완합니다.
   - **적용 가능성**: 초기 검색의 빠른 성능을 유지하면서, 검색 정확도를 크게 향상시킬 수 있습니다.

### 5. **Hybrid Retriever**
   - **설명**: Hybrid Retriever는 Sparse (예: BM25)와 Dense (예: DPR, ColBERT) 검색 모델을 결합하여 두 가지 방식의 장점을 모두 활용합니다. Sparse 모델은 단어 기반의 매칭에 강점을 가지며, Dense 모델은 의미 기반의 매칭에 유리합니다.
   - **적용 가능성**: 문서의 언어적 다양성과 복잡성 때문에 하나의 검색 모델로는 만족스러운 성과를 내기 어려운 경우에 유용합니다.

### 6. **RAG (Retrieval-Augmented Generation)**
   - **설명**: RAG는 검색과 생성 모델을 결합한 접근 방식으로, 검색된 문서를 기반으로 자연어 생성 모델이 답변을 생성하는 구조입니다. 이는 질의응답 시스템에서 효과적입니다.
   - **적용 가능성**: 검색된 문서에 대한 설명이나 요약이 필요한 경우 유용하며, 검색된 정보를 바탕으로 자연스러운 텍스트를 생성할 수 있습니다.

### 7. **Multiview Retriever**
   - **설명**: Multiview Retriever는 문서의 다양한 측면(예: 텍스트, 메타데이터 등)을 활용하여 검색을 수행하는 방법입니다. 각 뷰에서의 검색 결과를 조합하여 최종 결과를 도출합니다.
   - **적용 가능성**: 문서에 여러 가지 중요한 측면이 존재하는 경우, 이러한 다양한 정보를 활용하여 검색의 정확도를 높일 수 있습니다.

### 8. **Contextualized Query Expansion**
   - **설명**: 쿼리를 확장하여 다양한 문맥에서의 검색을 수행하는 방법입니다. BERT와 같은 모델을 사용하여 쿼리를 확장하면, 보다 다양한 문서가 검색될 수 있습니다.
   - **적용 가능성**: 쿼리가 간단하거나, 사용자가 예상치 못한 형태로 정보가 표현되어 있을 때 유용합니다.

### 9. **Contrastive Learning for Information Retrieval**
   - **설명**: 대조 학습(Contrastive Learning)은 검색 모델이 유사한 쌍을 가깝게, 그리고 비유사한 쌍을 멀리 떨어뜨리도록 학습하는 방법입니다. 검색 결과의 정밀도를 높이는 데 효과적입니다.
   - **적용 가능성**: 검색 결과의 세밀한 차이를 구분해야 하는 경우, 정확도를 크게 향상시킬 수 있습니다.

### 결론
정확한 정보 검색이 중요한 상황에서는 다양한 검색 기술과 알고리즘을 조합한 앙상블 접근이 매우 효과적일 수 있습니다. 위에서 언급한 EnsembleRetriever, ColBERT, DPR, Hybrid Retriever 등은 특히 유용할 수 있습니다. 상황에 따라 여러 방법을 실험적으로 조합하여 최적의 검색 성능을 도출하는 것이 중요합니다. 최신 연구를 기반으로 한 조합 방식을 시도하여 검색 정확도를 극대화할 수 있습니다.